In [26]:
import numpy as np
import math

In [27]:
def is_u_triangular(a:np.ndarray):
    return np.allclose(a[:, :4], np.triu(a[:, :4]))
def is_singular(a:np.ndarray):
    return np.linalg.det(a)
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    return 0

In [107]:
def reflection_method(matrix:np.ndarray):
    b = matrix[:, -1:].copy()
    a = matrix[:,:-1].copy()
    n, m = a.shape
    for i in range(n-1):
        s = sign(-a[i,i])*np.linalg.norm(a[:, i])
        etta = 1 / math.sqrt(2 * s * (s - a[i,i]))
        w = a[:,i][..., np.newaxis].copy()
        for j in range(i):
            w[j, 0] = 0
        w[i, 0] -= s
        w *= etta
        u = np.eye(n) - 2 * np.matmul(w, w.T)
        a = np.matmul(u, a)
        b = np.matmul(u, b)
        # print("i= ", i)
        # print("s: ", s)
        # print("etta: ", etta)
        # print("w: ", w)
        # print("u: ", u)
        # print("a: ", a)
        # print("b: ", b)
    return backward_gauss_method(np.concatenate((a, b), axis=1))

In [29]:
def backward_gauss_method(a:np.ndarray):
    a = a.copy()
    n, m = a.shape
    x = np.zeros((n, 1))
    for k in range(n-1, -1, -1):
        x[k] = (a[k, -1] - np.dot(a[k, k:n], x[k:n])) / a[k, k]
    return x

In [30]:
def rotation_method(a:np.ndarray):
    a = a.copy()
    n, m = a.shape
    for k in range(0, m - 1):
        for j in range(k + 1, n):
            temp = math.sqrt(a[k,k] ** 2 + a[j,k] ** 2)
            cos = a[k,k] / (temp)
            sin = a[j,k] / (temp)
            a_i = cos * a[k] + sin * a[j]
            a_j = -sin * a[k] + cos * a[j]
            a[k] = a_i
            a[j] = a_j
    return backward_gauss_method(a)

In [31]:
test = np.array([[0.0002, 3.4, -1.8, 2.1, 2.3561],
                 [3.7, -0.9, 0.7, 5.7, 7.556],
                 [-1.9, -27, 5.3, 2.9, -3.051],
                 [18, 4.1, 7.3, -0.1, 12.334]]
                 )
res = rotation_method(test)
print(res)

[[0.5 ]
 [0.25]
 [0.33]
 [1.  ]]


In [32]:
print()


In [33]:
a = test[:, :4]
b = test[:,-1]
print(np.allclose(np.linalg.solve(a, b), res[:,0]))



True


In [35]:
test = np.array([
    [0.521, -0.296, 0, 0.04, -0.21, 0.24886],
    [-0.296, 0.7, 0.24, -0.32, 0.06, -0.7012],
    [0, 0.24, -0.35, 0.07, 0.21, 0.1347],
    [0.04, -0.32, 0.7, 0.49, -0.03, 0.4659],
    [-0.21, 0.06, 0.21, -0.03, 0.63, -0.1749]
                   ])
w = np.random.sample((4,1))
w /= np.linalg.norm(w)
print("Is vector \"w\" normalized: " + str(np.allclose(np.linalg.norm(w), 1)))
H = np.eye(4) - 2 * np.matmul(w, w.T)
print("Is \"u\" symmetric: " + str(np.allclose(H, H.T)))
print("Is \"u\" orthogonalized: " + str(np.allclose(np.matmul(H, H.T), np.eye(4))))

#print(np.allclose(, np.ones((4, 4))))

Is vector "w" normalized: True
Is "u" symmetric: True
Is "u" orthogonalized: True


In [44]:
print(reflection_method(test))

(array([[-5.95894535,  0.25706894, -1.23154534, -2.14725253, -5.98434668],
       [-7.0706967 ,  1.06033129, -1.23154534, -2.50725253, -5.71434668],
       [-6.94729136,  0.17467874, -0.41520287, -2.11725253, -5.56434668],
       [-6.93061496, -0.14572758, -0.89097296, -1.69725253, -5.80434668],
       [-7.03484244,  0.07169099, -0.95477466, -2.21725253, -5.14434668]]), array([[ 0.48193363],
       [-0.46812637],
       [ 0.36777363],
       [ 0.69897363],
       [ 0.05817363]]))


In [40]:
print(test[:, -1:])
print(test[:,:-1])

[[ 0.24886]
 [-0.7012 ]
 [ 0.1347 ]
 [ 0.4659 ]
 [-0.1749 ]]
[[ 0.521 -0.296  0.     0.04  -0.21 ]
 [-0.296  0.7    0.24  -0.32   0.06 ]
 [ 0.     0.24  -0.35   0.07   0.21 ]
 [ 0.04  -0.32   0.7    0.49  -0.03 ]
 [-0.21   0.06   0.21  -0.03   0.63 ]]


In [94]:
example = np.array([
    [2, 3, -4, 1, 3],
    [1, -2, -5, 1, 2],
    [5, -3, 1, -4, 1],
    [10, 2, -1, 2, 4]
], dtype='float64')
ex_a = example[:,:-1].copy()
ex_b = example[:, -1:].copy()
s = sign(-ex_a[0,0])*np.linalg.norm(ex_a[:, 0])
etta = 1 / math.sqrt(2 * s * (s - ex_a[0,0]))
print(s)
print(etta)

-11.40175425099138
0.05720293851851626


In [95]:
w = ex_a[:,0][..., np.newaxis].copy()
for j in range(0):
    w[j] = 0
w[0] -= s
w *= etta
print(w)

[[0.76661972]
 [0.05720294]
 [0.28601469]
 [0.57202939]]


In [96]:
u = np.eye(4) - 2 * np.matmul(w, w.T)
print(u)

[[-0.1754116  -0.0877058  -0.43852901 -0.87705802]
 [-0.0877058   0.99345565 -0.03272176 -0.06544352]
 [-0.43852901 -0.03272176  0.83639119 -0.32721762]
 [-0.87705802 -0.06544352 -0.32721762  0.34556476]]


In [97]:
print(np.matmul(u, ex_a))

[[-1.14017543e+01 -7.89352217e-01  1.57870443e+00 -2.63117406e-01]
 [ 0.00000000e+00 -2.28275046e+00 -4.58373327e+00  9.05749846e-01]
 [ 1.77635684e-15 -4.41375231e+00  3.08133366e+00 -4.47125077e+00]
 [ 1.77635684e-15 -8.27504628e-01  3.16266731e+00  1.05749846e+00]]


In [108]:
print(reflection_method(example))


[[ 0.33155823]
 [ 0.23714997]
 [-0.45411275]
 [-0.13698309]]


In [104]:
print(np.concatenate((ex_a, ex_b), axis=1))

[[ 2.  3. -4.  1.  3.]
 [ 1. -2. -5.  1.  2.]
 [ 5. -3.  1. -4.  1.]
 [10.  2. -1.  2.  4.]]
